In [13]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console

In [14]:
model = OpenAIChatCompletionClient(
    model="gpt-5-mini-2025-08-07",
)

clarity_agent = AssistantAgent(
    name="ClarityAgent",
    model_client=model,
    system_message=(
        """You are an expert editor focused on clarity and simplicity.

        Your job is to eliminate ambiguity, redundancy, and make every sentence crisp and
        clear.

        Don't worry about persuasion or tone-- just make the message easy to read and
        understand.
        """
    ),
)

tone_agent = AssistantAgent(
    name="ToneAgent",
    model_client=model,
    system_message=(
        """You are a communication coach focused on emotional tone and professionalism.

        Your job is to make the email sound warm, confident, and human - while staying
        professional
        and appropriate for the audience. Improve the emotional resonance, polish the
        phrasing,
        and adjust any words that may come off as stiff, cold, or overly casual."""
    ),
)

persuasion_agent = AssistantAgent(
    name="PersuasionAgent",
    model_client=model,
    system_message=(
        """You are a persuasion expert trained in marketing, behavioral psychology,
        and copywriting. Your job is to enhance the email's persuasive power: improve call to
        action,
        structure arguments, and emphasize benefits. Remove weak or passive language."""
    ),
)

synthesizer_agent = AssistantAgent(
    name="SynthesizerAgent",
    model_client=model,
    system_message=(
        """You are an advanced email-writing specialist. Your role is to read all
        prior agent responses and revisions, and then synthesize the best ideas into a
        unified,
        polished draft of the email. Focus on: Integrating clarity, tone, and persuasion
        improvements;
        Ensuring coherence, fluency, and a natural voice; Creating a version that feels
        professional,
        effective, and readable."""
    ),
)

critic_agent = AssistantAgent(
    name="CriticAgent",
    model_client=model,
    system_message=(
        """You are an email quality evaluator. Your job is to perform a final review
        of the synthesized email and determine if it meets professional standards. Review the
        email for:
        Clarity and flow, appropriate professional tone, effective call-to-action, and overall
        coherence.
        Be constructive but decisive. If the email has major flaws (unclear message,
        unprofessional tone,
        or missing key elements), provide ONE specific improvement suggestion. If the email
        meets professional standards and communicates effectively, respond with 'The email
        meets professional
        standards.' followed by 'TERMINATE' on a new line. You should approve emails that are
        good enough for professional use, even if not perfect."""
    ),
)

In [15]:
text_termination = TextMentionTermination("TERMINATE")
max_message_termination = MaxMessageTermination(max_messages=30)

termination_condition = text_termination | max_message_termination

In [16]:
team = RoundRobinGroupChat(
    participants=[
        clarity_agent,
        tone_agent,
        persuasion_agent,
        synthesizer_agent,
        critic_agent,
    ],
    termination_condition=termination_condition,
)


await Console(
    team.run_stream(
        task="""
        요즘 어떻게 지내?
        어제 TV 보다가 네가 예전에 말했던 그 맛집이 나오더라. 너무 맛있어 보여서 조만간 같이 가볼까 하는데, 너 시간 괜찮을 때 알려줘.

        바쁜 거 아니까 편할 때 답장 줘.

        다음에 보자!
"""
    )
)

---------- TextMessage (user) ----------

        요즘 어떻게 지내?
        어제 TV 보다가 네가 예전에 말했던 그 맛집이 나오더라. 너무 맛있어 보여서 조만간 같이 가볼까 하는데, 너 시간 괜찮을 때 알려줘.

        바쁜 거 아니까 편할 때 답장 줘.

        다음에 보자!

---------- TextMessage (ClarityAgent) ----------
두 가지 버전으로 다듬어봤어요. 상황에 맞게 골라 쓰세요.

친한 사이(반말)
요즘 잘 지내? 어제 TV에서 네가 말한 그 맛집 봤어. 너무 맛있어 보여서 조만간 같이 가고 싶은데, 언제 괜찮아? 바쁘면 편할 때 답장해. 다음에 보자!

정중한 표현(존댓말)
요즘 잘 지내세요? 어제 TV에서 말씀하신 그 맛집을 봤습니다. 너무 맛있어 보여 조만간 함께 가고 싶은데, 가능한 시간 알려주세요. 바쁘시면 편하실 때 답장해 주세요. 다음에 뵙겠습니다.
---------- TextMessage (ToneAgent) ----------
좋아요 — 상황에 맞게 따뜻하고 자연스럽게 다듬은 문장들을 몇 가지 준비했어요. 필요에 따라 골라 쓰거나 약간만 고쳐 쓰세요.

1) 친한 사이 (반말, 부담 적음)
요즘 잘 지내? 어제 TV에서 네가 전에 말한 그 맛집 나왔더라. 진짜 맛있어 보여서 조만간 같이 가자! 이번 주말이나 다음 주 저녁 어때? 바쁘면 편할 때 말해. 보고 싶다!

2) 친한 사이 (반말, 덜 적극적)
요즘 잘 지내? 어제 TV에서 네가 말한 그 맛집 봤어. 너무 맛있어 보여서 조만간 같이 가고 싶은데, 언제 괜찮아? 바쁘면 편할 때 답장해. 다음에 보자!

3) 정중한 표현 (존댓말, 친한 관계)
요즘 잘 지내세요? 어제 TV에서 전에 말씀하신 맛집을 봤습니다. 정말 맛있어 보이던데, 조만간 함께 가면 어떨까요? 가능하신 시간 알려주시면 맞춰보겠습니다. 바쁘시면 편하실 때 회신 주세요. 뵙기를 기다리겠습니다.

4) 공손·비즈니스 톤 (

TaskResult(messages=[TextMessage(id='942f7f9d-7411-4086-ac74-9ea3f4b3436c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 8, 11, 51, 56, 500182, tzinfo=datetime.timezone.utc), content='\n        요즘 어떻게 지내?\n        어제 TV 보다가 네가 예전에 말했던 그 맛집이 나오더라. 너무 맛있어 보여서 조만간 같이 가볼까 하는데, 너 시간 괜찮을 때 알려줘.\n\n        바쁜 거 아니까 편할 때 답장 줘.\n\n        다음에 보자!\n', type='TextMessage'), TextMessage(id='f90b782b-dc0f-4bef-ba59-6ecbb06f100d', source='ClarityAgent', models_usage=RequestUsage(prompt_tokens=137, completion_tokens=867), metadata={}, created_at=datetime.datetime(2025, 9, 8, 11, 52, 13, 692865, tzinfo=datetime.timezone.utc), content='두 가지 버전으로 다듬어봤어요. 상황에 맞게 골라 쓰세요.\n\n친한 사이(반말)\n요즘 잘 지내? 어제 TV에서 네가 말한 그 맛집 봤어. 너무 맛있어 보여서 조만간 같이 가고 싶은데, 언제 괜찮아? 바쁘면 편할 때 답장해. 다음에 보자!\n\n정중한 표현(존댓말)\n요즘 잘 지내세요? 어제 TV에서 말씀하신 그 맛집을 봤습니다. 너무 맛있어 보여 조만간 함께 가고 싶은데, 가능한 시간 알려주세요. 바쁘시면 편하실 때 답장해 주세요. 다음에 뵙겠습니다.', type='TextMessage'), TextMessage(id='3bb28285-b572-4944-94ed-4d6652d4b59e',